In [45]:
# Load filter2D + dilate overlay
from pynq import Overlay
bareHDMI = Overlay("/usr/local/lib/python3.6/dist-packages/"
               "pynq_cv/overlays/xv2Filter2DDilate.bit")
import pynq_cv.overlays.xv2Filter2DDilate as xv2

# Load xlnk memory mangager
from pynq import Xlnk
Xlnk.set_allocator_library("/usr/local/lib/python3.6/dist-packages/"
                           "pynq_cv/overlays/xv2Filter2DDilate.so")
mem_manager = Xlnk()

#hdmi_in = bareHDMI.video.hdmi_in
hdmi_out = bareHDMI.video.hdmi_out

In [46]:
from pynq.lib.video import *
Mode = VideoMode(640,480,24)
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

In [47]:
import cv2
import numpy as np

videoIn = cv2.VideoCapture(-1)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("capture device is open: " + str(videoIn.isOpened()))
#frame_1080p = np.zeros((1080,1920,3)).astype(np.uint8)
ret, reference = videoIn.read()
reference_grey = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY)
reference_blur = cv2.GaussianBlur(reference_grey, (5, 5), 0)

#frame_1080p[0:480,0:640,:] = reference[0:480,0:640,:]
#hdmi_out.writeframe(reference)

capture device is open: True


array([[[ 68,  88, 113],
        [144, 162, 185],
        [193, 211, 228],
        ..., 
        [219, 211, 211],
        [221, 211, 211],
        [220, 210, 210]],

       [[ 69,  86, 112],
        [142, 158, 181],
        [192, 208, 225],
        ..., 
        [216, 206, 206],
        [215, 205, 205],
        [217, 205, 205]],

       [[ 72,  85, 111],
        [139, 151, 175],
        [189, 203, 221],
        ..., 
        [214, 202, 202],
        [213, 201, 201],
        [214, 200, 201]],

       ..., 
       [[157, 146, 138],
        [156, 145, 137],
        [155, 144, 136],
        ..., 
        [240, 240, 240],
        [240, 240, 240],
        [240, 240, 240]],

       [[157, 147, 137],
        [156, 146, 136],
        [155, 145, 135],
        ..., 
        [240, 240, 240],
        [240, 240, 240],
        [240, 240, 240]],

       [[157, 147, 137],
        [156, 146, 136],
        [155, 145, 135],
        ..., 
        [240, 240, 240],
        [240, 240, 240],
        [240, 240,

In [48]:

outframe = hdmi_out.newframe()
outframe[0:480,0:640,:] = reference[0:480,0:640,:]
hdmi_out.writeframe(outframe)

In [49]:
import time

outframe = hdmi_out.newframe()
numframes = 2000    
kernelVoid = np.zeros(0)

start = time.time()    
for _ in range(numframes):
    ret, frame_vga = videoIn.read()

    if (ret):
       
        grey = cv2.cvtColor(frame_vga, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grey, (5, 5), 0)
        difference = cv2.absdiff(reference_blur, blur)
        threshold = cv2.threshold(difference, 85, 255, cv2.THRESH_BINARY)[1]
        dilated = cv2.dilate(threshold, kernelVoid, iterations=2)
        _, contours, hier = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for i in contours:
            contourArea = cv2.contourArea(i)
            if contourArea < 17500 or contourArea > 35000:
                continue
            (x, y, w, h) = cv2.boundingRect(i)
           
            cv2.rectangle(frame_vga, (x, y), (x + w, y + h), (0, 0, 255), 2) 
            outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
            hdmi_out.writeframe(outframe) 
            #print("after out frame")
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))
hdmi_out.close()
videoIn.release()

Frames per second:  10.97609637442543


Frames per second:  2.1918136134859654
